In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.cuda
import torchvision.transforms.functional as TF
import torchvision.transforms as transforms
import os
from joblib import dump, load
from PIL import Image
from tqdm.notebook import tqdm
from torch.nn.functional import relu
from torch.utils.data import DataLoader, Dataset
from sklearn.manifold import TSNE
from SHG import SHG
from utils import *
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.base import clone
from sklearn_extra.cluster import KMedoids
from AE import CONV_AE

In [5]:
""" PATHS """

# MODELS
AE_MODEL_ALL_PATH = "C:/Users/André/OneDrive 2/OneDrive/Skrivebord/bsc_data/latent_space_data/models/all/model.pth"
AE_MODEL_FULL_PATH = "C:/Users/André/OneDrive 2/OneDrive/Skrivebord/bsc_data/latent_space_data/models/full/model.pth"

# DATA
ALL_PATH = "C:/Users/André/OneDrive 2/OneDrive/Skrivebord/bsc_data/latent_space_data/train_all/"
FULL_PATH = "C:/Users/André/OneDrive 2/OneDrive/Skrivebord/bsc_data/latent_space_data/train_full_skeleton/"

# SAVING PATH
ALL_SAVING_PATH = "C:/Users/André/OneDrive 2/OneDrive/Skrivebord/bsc_data/latent_space_data/AE_latent_spaces/all/"
FULL_SAVING_PATH = "C:/Users/André/OneDrive 2/OneDrive/Skrivebord/bsc_data/latent_space_data/AE_latent_spaces/full/"

In [8]:
""" DATASET AND DATALOADER """

class dataset(Dataset):
    def __init__(self, PATH):
        self.FULL_PATH = PATH
        self.listdir = os.listdir(self.FULL_PATH)

    def __len__(self):
        return len(self.listdir)

    def __getitem__(self, i):
        ID = self.listdir[i]
        path = self.FULL_PATH + ID
        data = np.load(path)
        return data, ID

ALL_DATASET = dataset(ALL_PATH)
ALL_DATALOADER = DataLoader(ALL_DATASET)

FULL_DATASET = dataset(FULL_PATH)
FULL_DATALOADER = DataLoader(FULL_DATASET)

In [6]:
""" Configurations """
# DEVICE
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# MODELS
AE_ALL = CONV_AE().to(device)
AE_ALL.load_state_dict(torch.load(AE_MODEL_ALL_PATH))
AE_ALL.eval()

AE_FULL = CONV_AE().to(device)
AE_FULL.load_state_dict(torch.load(AE_MODEL_FULL_PATH))
AE_FULL.eval()

CONV_AE(
  (encoder): Sequential(
    (0): Conv2d(256, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(192, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): ReLU(inplace=True)
  )
  (bottleneck): Sequential(
    (0): Linear(in_features=1024, out_features=1024, bias=True)
    (1): ReLU(inplace=True)
  )
  (decoder): Sequential(
    (0): ConvTranspose2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): ConvTranspose2d(128, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): ConvTranspose2d(192, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): ReLU(inplace=True)
  )
)

In [25]:
""" SAVING PREDICTIONS OF AE_ALL """
with torch.no_grad():
    for x, ID in tqdm(ALL_DATALOADER, leave = False):
        ID = ID[0]
        x = x.to(device, dtype = torch.float)
        y = AE_ALL.encode(x).cpu().data.numpy()
        np.save(ALL_SAVING_PATH + ID, y)

In [9]:
""" SAVING PREDICTIONS OF AE_FULL """
with torch.no_grad():
    for x, ID in tqdm(FULL_DATALOADER, leave = False):
        ID = ID[0]
        x = x.to(device, dtype = torch.float)
        y = AE_FULL.encode(x).cpu().data.numpy()
        np.save(FULL_SAVING_PATH + ID, y)